<a href="https://colab.research.google.com/github/avkaz/DeepLearningPetIdentification/blob/fixes/finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading utility file and importing need dependesies

In [18]:

import requests
# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)



In [19]:
!pip install faiss-cpu
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import requests
import json
import pandas as pd
import faiss
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from google.colab import files
import utility
from itertools import islice


## Getting Data

In [20]:
google_drive_url = "https://drive.google.com/file/d/1VR5GWGrVjEtJHEzTPIB-EHDQMG3UnmZ9/view?usp=sharing"
download_file_from_google_drive(google_drive_url, "./data/pets_pair.json")

TypeError: download_file_from_google_drive() takes 0 positional arguments but 2 were given

In [3]:
data_in = utility.load_json_and_transform_lists_to_tensors('')

In [4]:
data = dict(list(data_in.items())[:200])

def functions

In [5]:

def filter_pets_by_images(data, min_images=3):
    """Filters pets with at least 'min_images' images."""
    return {key: value for key, value in data.items() if len(value['images']) >= min_images}


def create_model(input_dim):
    """Create a model that processes embeddings directly (e.g., from EfficientNetB0)."""
    inputs = tf.keras.Input(shape=(input_dim,))  # Input layer for the flattened embedding vector
    x = layers.Dense(256, activation='relu')(inputs)  # Dense layer to process embeddings
    x = layers.Dense(128, activation='relu')(x)  # Another dense layer for more abstraction
    output = layers.Dense(1, activation='sigmoid')(x)  # Output layer
    model = Model(inputs, output)
    return model

def generate_embeddings(model, image_tensors):
    """Generate embeddings for the list of image URLs."""
    embeddings = []
    for img_tensor in image_tensors:
        img = img_tensor
        emb = model.predict(np.expand_dims(img, axis=0))
        embeddings.append(emb.flatten())
    return np.array(embeddings)
@tf.function
def triplet_loss(y_true, y_pred, margin=1.0):
    """Triplet loss function."""
    # Anchor, positive, and negative embeddings
    anchor, positive, negative = y_pred
    # Calculate the Euclidean distance
    positive_distance = tf.sqrt(tf.reduce_sum(tf.square(anchor - positive), axis=1))
    negative_distance = tf.sqrt(tf.reduce_sum(tf.square(anchor - negative), axis=1))
    # Calculate triplet loss
    return tf.reduce_mean(tf.maximum(positive_distance - negative_distance + margin, 0))


In [6]:
model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


only pets with 3+ images will be used

In [7]:
data= filter_pets_by_images(data)

In [9]:
print(dict(list(data_in.items())[:1]))

{'tanyny-chomutov-2024-12-21': {'Jméno': 'Tanyny', 'Pohlaví': 'Samec', 'Kraj': 'Ústecký', 'Okres': 'Chomutov', 'Plemeno': 'Kříženec', 'Věk': '5 let', 'Barva': 'Černá', 'Velikost': 'Střední - 10-17kg', 'url': 'https://www.psidetektiv.cz/zvire/tanyny-chomutov-2024-12-21', 'images': ['https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190390.jpg', 'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190391.jpg', 'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190392.jpg', 'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190393.jpg', 'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190394.jpg']}}


embeddings

In [ ]:
# create a mapping of pet identifiers to integer indices
pet_ids = list(data.keys())
pet_to_idx = {pet_id: idx for idx, pet_id in enumerate(pet_ids)}

# generate embeddings for all pets and store them
embeddings_dict = {
    pet_key: generate_embeddings(model, pet_info['images'])
    for pet_key, pet_info in data.items()
}




1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 732ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 641ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655m

In [ ]:
def generate_triplet(pet_key, pet_info, embeddings_dict, data):
    embeddings = embeddings_dict[pet_key]  # use pre-generated embeddings

    # create triplets: anchor, positive, and negative
    for i in range(len(embeddings)):
        anchor = embeddings[i]

        # pick a random positive example from the same pet's observation, excluding the anchor image
        positive_idx = np.random.choice([idx for idx in range(len(embeddings)) if idx != i ])
        positive = embeddings[positive_idx]

        # pick a random negative example (different pet)
        negative_pet_key = np.random.choice([key for key in data if key != pet_key])
        negative_embedding = embeddings_dict[negative_pet_key][0]
        #print(i)
        #print(positive_idx)
        #print(negative_pet_key)
        yield [anchor, positive, negative_embedding], pet_to_idx[pet_key]
    #print(embeddings_dict[pet_key])


In [ ]:

# triplet training
triplets = []
labels = []

for pet_key, pet_info in data.items():
    for triplet, label in generate_triplet(pet_key, pet_info, embeddings_dict, data):
        triplets.append(triplet)
        labels.append(label)

# Convert triplets to numpy array
triplets = np.array(triplets)

# Flatten the embeddings for FAISS
flattened_triplets = np.array([
    np.concatenate([anchor.flatten(), positive.flatten(), negative.flatten()])
    for anchor, positive, negative in triplets
])


In [ ]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_triplets, labels, test_size=0.2, random_state=42)

# Initialize FAISS index for similarity search
embedding_dim = X_train.shape[1]  # The size of the flattened embeddings
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric for similarity search

# Add training data embeddings to the FAISS index
index.add(X_train)  # No need to reshape, as X_train is already flat


input_dim = 3840

# Create the model
model = create_model(input_dim)

# Compile and train the model
model.compile(optimizer='adam', loss=lambda y_true, y_pred: triplet_loss(y_true, y_pred, margin=1.0))
model.fit(X_train, np.array(y_train), epochs=60, batch_size=32)

Epoch 1/60


OperatorNotAllowedInGraphError: in user code:

    File "<ipython-input-27-310fb775cc8e>", line 27, in triplet_loss  *
        anchor, positive, negative = y_pred

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.
